In [1]:
import os

In [2]:
os.environ["DGLBACKEND"] = "pytorch"

In [51]:
import torch 
import pandas as pd
import dgl
from dgl.data import DGLDataset
import dgl.function as fn
import numpy as np

In [4]:
ds = dgl.data.CSVDataset('./DGL_playground/toy_dataset/')

Done loading data from cached files.


In [5]:
g = ds[0]

In [6]:
print(g)


Graph(num_nodes=10, num_edges=20,
      ndata_schemes={' na2': Scheme(shape=(), dtype=torch.int64), ' na1': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'ea2': Scheme(shape=(), dtype=torch.int64), 'ea1': Scheme(shape=(), dtype=torch.int64), 'w': Scheme(shape=(), dtype=torch.int64)})


In [119]:
g.edata['w'] = torch.rand(g.num_edges(),1)
g.edata['w']

tensor([[0.8746],
        [0.4106],
        [0.2491],
        [0.9596],
        [0.3840],
        [0.8732],
        [0.6992],
        [0.7689],
        [0.4988],
        [0.7450],
        [0.1181],
        [0.3896],
        [0.6899],
        [0.3884],
        [0.3385],
        [0.9596],
        [0.4583],
        [0.0301],
        [0.1538],
        [0.6074]])

In [8]:
#Create wealth per node/agent in the network as a node feature/attribute
g.ndata['k'] = torch.rand(g.num_nodes(),1)*1000

In [9]:
#Create savings propensity as a node features per agent
g.ndata['lambda'] = torch.rand(g.num_nodes(),1)

In [25]:
g.edges(form='uv')[0]

tensor([0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8])

In [26]:
g.ndata['n_neighbours'] = torch.zeros(g.num_nodes(),1)

In [31]:
for i in g.nodes():
    g.ndata['n_neighbours'][i] =  np.count_nonzero(g.edges(form='uv')[0] == i)


In [80]:
g.ndata['zeros'] = torch.zeros(g.num_nodes(),1)


In [81]:
g.ndata['ones'] = torch.ones(g.num_nodes(),1)

In [90]:
g.update_all(fn.u_add_v('zeros','ones','ones'), fn.sum('ones', 'tot_nbr'))

In [155]:
g.ndata['tot_wgt'] = torch.zeros(g.num_nodes(),1)
g.update_all(fn.u_add_e('tot_wgt','w','m'), fn.sum('m', 'tot_wgt'))
g.ndata['tot_wgt']

tensor([[2.6016],
        [1.5738],
        [0.7945],
        [1.0572],
        [0.8872],
        [1.1346],
        [0.6120],
        [0.9459],
        [0.9897],
        [0.0000]])

In [99]:
g.apply_edges(fn.e_div_u('w','tot_wgt','per_wgt'))

In [122]:
g.edata['w']

tensor([[0.8746],
        [0.4106],
        [0.2491],
        [0.9596],
        [0.3840],
        [0.8732],
        [0.6992],
        [0.7689],
        [0.4988],
        [0.7450],
        [0.1181],
        [0.3896],
        [0.6899],
        [0.3884],
        [0.3385],
        [0.9596],
        [0.4583],
        [0.0301],
        [0.1538],
        [0.6074]])

In [106]:
g.edges()

(tensor([0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8]),
 tensor([1, 2, 3, 0, 2, 0, 1, 0, 4, 5, 3, 5, 3, 4, 7, 8, 6, 8, 6, 7]))